In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import re

load_dotenv()

True

In [3]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [19]:
# Function to format the instruction from key facts and event summary
def generate_instruction_from_facts_and_summary(key_facts, event_summary):
    # Format the key facts into a bullet point list
    formatted_facts = '\n'.join(key_facts)

    # Construct the instruction template with the provided facts and event summary
    instruction = f"""
### Instruction:
Given the following facts:
{formatted_facts}

And the event described:
{event_summary}

Write a column that reflects on the broader implications of this event. Consider how it may affect public policy, societal attitudes, or future developments.
    """
    return instruction.strip()

# Main function to extract key facts and event summary, then format the instruction
def generate_instruction(article_content):
    # Prepare the conversation state for GPT-4o chat-based completion
    conversation_state = [
        {"role": "system", "content": "You are a helpful assistant tasked with extracting only factual key points and a factual summary of the main event from the following content. Do not mention or reference the original article in any way. Avoid phrases like 'The article states' or 'According to the text'. Present the key facts and the main event in a neutral, straightforward manner. Please format the response like this:\n\n**Key Facts**:\n- Fact 1\n- Fact 2\n- Fact 3\n\n**Factual Summary**:\n- [Brief summary of the main event]."},
        {"role": "user", "content": f"Here is the content:\n\n{article_content}\n\nPlease extract the following:\n1. Key facts (factual points that are mentioned explicitly).\n2. A factual summary of the main event (describe the central event in an objective and neutral way without adding opinions or subjective analysis)."}
    ]

    # Call GPT-4o chat-based completion API
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=conversation_state,
            max_tokens=600
        )

        # Extract the response from the assistant
        output = response.choices[0].message.content.strip()

        # Use case-insensitive flag via re.IGNORECASE, and clean up the expression
        key_facts_match = re.search(r"\*\*Key Facts\*\*([\s\S]*?)\*\*Factual Summary\*\*", output, re.IGNORECASE)
        event_summary_match = re.search(r"\*\*Factual Summary\*\*([\s\S]*)", output, re.IGNORECASE)
        
        if key_facts_match:
            # Extract key facts (assuming each fact starts with a hyphen "-")
            key_facts_section = key_facts_match.group(1)
            key_facts = [fact.strip() for fact in key_facts_section.split("\n") if fact.strip().startswith('-')]
            # key_facts = key_facts[:-2]
            print(key_facts)
        else:
            key_facts = []

        if event_summary_match:
            event_summary = event_summary_match.group(1).strip()

            # Remove any leading colon or unwanted characters before the summary
            event_summary = re.sub(r"^[:\-\s]+", "", event_summary).strip()
        else:
            event_summary = ""

        # Format the final instruction using the key facts and event summary
        instruction = generate_instruction_from_facts_and_summary(key_facts, event_summary)
        return instruction


    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    

# Example usage with one article's content
article_content = """
Alman medyasında Türkiye hakkında bir haber çıktığında hepimiz önce bir irkiliriz.
Acaba terör örgütlerinden hangisini öveceğini düşünürüz.
Bu Almanya'ya karşı bir önyargı değil, yılların getirdiği bir deneyimin sonucudur.
Alman medyası, PKK'dan DHKP/ C'ye, FETÖ'den MLKP'ye nerede Türkiye'nin başına bela olmuş bir terör örgütü varsa onu övmesi ve meşrulaştırmasıyla meşhurdur.
Ancak Alman medyasında Türkiye hakkında yer alan bir haber hepimizi biraz şaşırttı.
Haberi veren Spiegel dergisi, Almanya'nın Türkiye'ye torpido ve güdümlü füze ihracatı için izin verdiğini açıkladı.
Alman hükümetinde bir değişiklik olmadığı, paralel bir evrene de geçmediğimize göre bu oldukça önemli bir haberdi.
Çünkü Almanya merkezli başka bir haber de Siemens firmasının Akkuyu Nükleer Santrali'miz için gerekli ekipmanları yollamadığı gündemdeyken çok sayıda Türkiye düşmanı bakana sahip Alman hükümeti bu izni vermişti.
Cevap aslında oldukça basit ama bir o kadar da önemli: Çünkü artık biz üretiyoruz.
Türk savunma sanayii, Cumhurbaşkanı Erdoğan önderliğinde Türk mühendislerinin insanüstü çabasıyla yurtdışında paramızla bize verilmeyen hangi ürün varsa onu üretmek için var gücüyle çalışıp ambargolarını başlarına çaldı.
O yüzden artık istediğimiz ürünü vermediklerinde çok da dert etmiyoruz.
ABD ve İsrail'in bize vaktiyle İHA vermeyip Türkiye'yi dünyanın en büyük İHA üreticisi yapmalarında olduğu gibi onlarca örnek var.
Mesela Kanada, SİHA'larımız için Elektro-Optik Keşif, Gözetleme ve Hedef Tespit Sistemi CATS'ların satışını yasaklayarak bizi bu sektöre de sokmuştu.
Hatta artık Kanada'dan daha kaliteli ürünler geliştirmiştik.
Ve evet, Kanada da ""ambargosunu"" kaldırmıştı.
Bugün de Almanya'nın bize vaktiyle vermediği torpido ve güdümlü füzeleri üretmeye başladık.
Almanya o yüzden bu satışa izin vererek Türkiye'deki Alman hayranlarını şaşırttı.
Terör örgütü PKK ile mücadele ettiğimiz için parasıyla sattığı tankların parçalarını yollamayan, Türkiye'nin enerji bağımsızlığı için gerekli olan barajları engellemek için her türlü imkânını seferber eden Almanya'dan benzer haberler göreceğiz.
Bu zamana kadar vermek istemediği ne kadar savunma sanayii ürünü varsa şimdi satmak için kendisi çabalayacak.
Bu nokta bizi rehavete sevk etmemeli.
Elbette ki Türk Silahlı Kuvvetleri'mizin ihtiyaçlarının karşılanması gerekiyor ve bu yapılmalı.
Ancak satın aldığımız her ürünün daha iyisini kendi imkânlarımızla üretme amacımızdan bir an bile vazgeçmemeliyiz.
Biz bu hedeften sapmadığımız sürece düne kadar kırk dereden su getiren ""müttefiklerimizin"" kapımızı aşındırdığını göreceğiz.
Çünkü artık biz üretiyoruz.
"""

# Generate the instruction (key facts and event summary)
instruction = generate_instruction(article_content)
print(instruction)

['- Spiegel magazine reported that Germany has allowed the export of torpedoes and guided missiles to Turkey.', '- There were no changes in the German government when this decision was made.', "- There is a contrast between this decision and issues with other companies like Siemens, which reportedly did not send equipment for Turkey's Akkuyu Nuclear Power Plant.", "- Turkey's defense industry has been strengthening by producing its own defense products.", '- Canada previously banned the sale of Electro-Optical Reconnaissance, Surveillance and Target Acquisition Systems CATS but later lifted the embargo.', '- Germany had previously avoided sending certain defense products, including tank parts, to Turkey.']
:
- The main event is that Germany, through an announcement in Spiegel magazine, has approved the export of certain military products, specifically torpedoes and guided missiles, to Turkey, highlighting a shift in Germany's export policy towards Turkey despite a history of restrictio